### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [1]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [4]:
#Load doc to vec model
doc2vec_model_name = 'reviews.d2v'
doc2vec_model = Doc2Vec.load(doc2vec_model_name)

In [5]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is very bad video. I don't like it"))],
[doc2vec_model.infer_vector(extract_words("video sucks."))])

array([[0.7315774]], dtype=float32)

In [6]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_text_pos=0):
    sentences = []
    sentvecs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        with open(doc_dir + "/" + fname, encoding='UTF-8') as f:
            print("files being read : {0}".format(fname))
            for i, line in enumerate(f):
                line_split = line.strip().split('\t')
                sentences.append(line_split[comment_text_pos])
                words = extract_words(line_split[comment_text_pos])
                sentvecs.append(model.infer_vector(words, steps=10)) # create a vector for this document
        
    #sentences, sentvecs together
    return sentences, sentvecs

In [7]:
doc_dir = "data/Test"
sentences, sentvecs = get_doc2vec(doc2vec_model, doc_dir)

files being read : yelp_labelled.txt


In [8]:
print("sentences : {0}".format(len(sentences)))

sentences : 1000


In [9]:
sentences[0:5]

['Wow... Loved this place.',
 'Crust is not good.',
 'Not tasty and the texture was just nasty.',
 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.',
 'The selection on the menu was great and so were the prices.']

In [10]:
sentvecs[0:5]

[array([ 0.2580039 ,  0.17749394,  0.16608469, -0.04063711, -0.23091166,
        -0.21782547,  0.18073712, -0.4600048 ,  0.20689465,  0.3588468 ,
         0.17917204, -0.17277159, -0.05354812,  0.01287622, -0.03258529,
         0.01657632,  0.04913678,  0.05231582, -0.17856082,  0.1171497 ,
        -0.24506575, -0.17292798, -0.21647501,  0.01017446,  0.17202368,
        -0.01229254,  0.08021051,  0.27074   , -0.21875386,  0.03605529,
         0.24077706, -0.1612347 ,  0.16099921,  0.03200743,  0.02535386,
        -0.13572364,  0.05821555, -0.05677211,  0.26938176,  0.12989739,
        -0.07208392, -0.08137994, -0.01896521, -0.0694478 ,  0.27666095,
         0.06179665,  0.14333087,  0.09620525, -0.33399186,  0.05078559],
       dtype=float32),
 array([-0.00849153,  0.15919824,  0.01353165, -0.00815025,  0.06872298,
        -0.16078253,  0.06390469, -0.07502481,  0.0244438 ,  0.02509074,
        -0.09995453, -0.11003245, -0.15937136,  0.21890596,  0.01556243,
        -0.07222971, -0.010

In [13]:
trained_clf = 'doc2vec_spam_rf.model'
loaded_clf = joblib.load(trained_clf)

In [14]:
y_pred = loaded_clf.predict(sentvecs)
print(y_pred)

[0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0
 1 1 0 0 0 0 1 0 0 0 0 0 

In [16]:
trained_clf = 'doc2vec_spam_kn.model'
loaded_clf = joblib.load(trained_clf)

In [14]:
y_pred = loaded_clf.predict(sentvecs)
print(y_pred)

[1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0
 0 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0
 0 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 1
 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1
 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1
 1 0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0
 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 0
 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1
 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1
 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1
 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1
 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 0
 0 1 1 0 1 1 1 0 1 0 1 0 